Code này chạy Julia

20221211 Đăng cài Julia bản 1.8.3, sau đó chạy Julia, trong môi trường Julia thì dùng các lệnh sau để cài đặt:

using Pkg

Pkg.add("IJulia") # tạo kết nối giữa Julia backend với Jupyter

Pkg.add("JuMP") # optimization toolbox (https://jump.dev/JuMP.jl/stable/installation/)

Pkg.add("AmplNLWriter") # to use solvers with AMPL interface https://github.com/jump-dev/AmplNLWriter.jl

Pkg.add("Bonmin_jll") # optimization solver Bonmin

Pkg.add("Couenne_jll") # solver Couenne

Pkg.add("Cbc") # solver Cbc

Pkg.add("Clp") # solver Clp

Pkg.add("ECOS") # solver ECOS

Pkg.add("GLPK") # solver GLPK

Pkg.add("HiGHS") # solver HiGHS

Pkg.add("Ipopt") # solver Ipopt

Pkg.add("SCIP") # solver SCIP

Pkg.add("Juniper") # solver Juniper

Pkg.add("OSQP") # solver OSQP

Pkg.add("NLopt") # solver NLopt

Pkg.add("DataFrames") # to deal with data in examples

Pkg.add("CSV") # to deal with CSV data in examples

Pkg.add("Plots") # plotting



In [1]:
1 + pi

4.141592653589793

In [2]:
2^3 # comment đây là lũy thừa

8

# Test JuMP with an example: https://jump.dev/JuMP.jl/stable/tutorials/linear/diet/#The-diet-problem

In [3]:
using JuMP
import HiGHS
import DataFrames


In [4]:
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [5]:
foods = DataFrames.DataFrame(
    [
        "hamburger" 2.49 410 24 26 730
        "chicken" 2.89 420 32 10 1190
        "hot dog" 1.50 560 20 32 1800
        "fries" 1.89 380 4 19 270
        "macaroni" 2.09 320 12 10 930
        "pizza" 1.99 320 15 12 820
        "salad" 2.49 320 31 12 1230
        "milk" 0.89 100 8 2.5 125
        "ice cream" 1.59 330 8 10 180
    ],
    ["name", "cost", "calories", "protein", "fat", "sodium"],
)

Row,name,cost,calories,protein,fat,sodium
,Any,Any,Any,Any,Any,Any
1,hamburger,2.49,410,24,26,730
2,chicken,2.89,420,32,10,1190
3,hot dog,1.5,560,20,32,1800
4,fries,1.89,380,4,19,270
5,macaroni,2.09,320,12,10,930
6,pizza,1.99,320,15,12,820
7,salad,2.49,320,31,12,1230
8,milk,0.89,100,8,2.5,125
9,ice cream,1.59,330,8,10,180


In [6]:
limits = DataFrames.DataFrame(
    [
        "calories" 1800 2200
        "protein" 91 Inf
        "fat" 0 65
        "sodium" 0 1779
    ],
    ["name", "min", "max"],
)

Row,name,min,max
,Any,Any,Any
1,calories,1800,2200
2,protein,91,Inf
3,fat,0,65
4,sodium,0,1779


In [7]:
@variable(model, x[foods.name] >= 0);


In [8]:
@objective(
    model,
    Min,
    sum(food["cost"] * x[food["name"]] for food in eachrow(foods)),
);

In [9]:
for limit in eachrow(limits)
    intake = @expression(
        model,
        sum(food[limit["name"]] * x[food["name"]] for food in eachrow(foods)),
    )
    @constraint(model, limit.min <= intake <= limit.max)
end

In [10]:
print(model)

In [11]:
optimize!(model)
solution_summary(model)

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
4 rows, 9 cols, 36 nonzeros
4 rows, 9 cols, 36 nonzeros
Presolve : Reductions: rows 4(-0); columns 9(-0); elements 36(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(814) 0s
          6     1.1828861111e+01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  1.1828861111e+01
HiGHS run time      :          0.00


* Solver : HiGHS

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution
  Objective value      : 1.18289e+01
  Objective bound      : 0.00000e+00
  Relative gap         : Inf
  Dual objective value : 1.18289e+01

* Work counters
  Solve time (sec)   : 7.48873e-04
  Simplex iterations : 6
  Barrier iterations : 0
  Node count         : -1


In [12]:
for food in foods.name
    println(food, " = ", value(x[food]))
end

hamburger = 0.6045138888888871
chicken = 0.0
hot dog = 0.0
fries = 0.0
macaroni = 0.0
pizza = 0.0
salad = 0.0
milk = 6.9701388888888935
ice cream = 2.5913194444444447


# Switch to another solver for the model and try to solve again

In [13]:
using AmplNLWriter, Bonmin_jll

set_optimizer(model, () -> AmplNLWriter.Optimizer(Bonmin_jll.amplexe)) # note that syntax to set optimizer based on AmplNLWriter is different to a solver that is interfaced directly to JuMP
optimize!(model)
for food in foods.name
    println(food, " = ", value(x[food]))
end

Bonmin 1.8.8 using Cbc 2.10.5 and Ipopt 3.14.4
bonmin: 
Cbc3007W No integer variables - nothing to do

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 11.828861       14 0.027724
Cbc3007W No integer variables - nothing to do

 	"Finished"
hamburger = 0.6045141199666294
chicken = -9.97078373783885e-9
hot dog = -9.981653157313894e-9
fries = -9.984424574497262e-9
macaroni = -9.99251944383328e-9
pizza = -9.987269834606244e-9
salad = -9.977258028238156e-9
milk = 6.9701383245810495
ice cream = 2.5913193439887703


In [14]:
import Clp
set_optimizer(model, Clp.Optimizer)
optimize!(model)
for food in foods.name
    println(food, " = ", value(x[food]))
end

hamburger = 0.6045138888888892
chicken = 0.0
hot dog = 0.0
fries = 0.0
macaroni = 0.0
pizza = 0.0
salad = 0.0
milk = 6.970138888888888
ice cream = 2.591319444444444
Coin0506I Presolve 4 (0) rows, 9 (0) columns and 36 (0) elements
Clp0006I 0  Obj 0 Primal inf 6.0580355 (2)
Clp0006I 6  Obj 11.828861
Clp0000I Optimal - objective value 11.828861
Clp0032I Optimal objective 11.82886111 - 6 iterations time 0.002


In [ ]:
import SCIP

set_optimizer(model, SCIP.Optimizer)
optimize!(model)
for food in foods.name
    println(food, " = ", value(x[food]))
end

In [ ]:
#reduced_cost(x)